In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case1,Case2,Case3
import numpy as np
import math
import xlwt
from pathlib import Path

In [3]:

BASE_DIR = Path().absolute()
DIR = BASE_DIR/'Results'
DIR


WindowsPath('c:/Users/fan_shuyuan/Desktop/科研ing/工艺流程优化/ASBO-V6/Results')

In [4]:
*

SyntaxError: invalid syntax (796671574.py, line 1)

In [ ]:
sol = 'ASBO'
case_name = 'Case3'
runs_number = 100
list_realvalue = []

def func(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3(x)
    cons = [c1,c2,c3,c4]
    return -obj,cons

nc_lb = np.array([-1,-1,-2,0,])
nc_ub = np.array([2,  4, 3,1.5])
cons = NonlinearConstraint(func, nc_lb, nc_ub)
pbounds = {'x1': (-5, 12),'x2': (-20, 22),'x3': (-10, 10),'x4': (-12, 20),'x5': (-15, 18),}
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)

for i in range(runs_number):
    optimizer = BayesianOptimization(f=func, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
    optimizer.maximize(init_points=100, n_iter=200,p_hyper=[5, 2, 0.78, 0.95,1e-5],acquisition_function=acquisition_function)
    print(optimizer.max)
    list_realvalue.extend([optimizer.max['target']])
    f = xlwt.Workbook('encoding = utf-8')
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
    for i in range(len(list_realvalue)):
        res = list_realvalue[i]
        sheet1.write(i,0,res)
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')


{'target': -1.0696983468467463, 'params': {'x1': 4.870232103568217e-05, 'x2': 7.135390869077265, 'x3': -6.185766178162072, 'x4': -2.5863292282461336, 'x5': 2.2113205539751086}, 'constraint': array([-0.37137284,  1.33465259,  1.00876478,  0.84640234])}
{'target': -1.0697737143988744, 'params': {'x1': -7.533000743562435e-05, 'x2': 14.2972863369797, 'x3': -4.401750647208934, 'x4': 11.212459240003856, 'x5': 4.636357115342177}, 'constraint': array([0.56921873, 0.29101443, 0.99993105, 0.34277624])}
{'target': -3.6136821457868895, 'params': {'x1': 0.8898466695935525, 'x2': -3.8866472952107936, 'x3': -5.3432424630755735, 'x4': -1.6940768942806856, 'x5': 7.5314724564592455}, 'constraint': array([ 0.71302896, -0.00664406,  0.1093133 ,  1.23684019])}
{'target': -1.072614936610695, 'params': {'x1': -0.0010733396151389714, 'x2': -2.4252047529036695, 'x3': 5.967251463864597, 'x4': 10.851647577314862, 'x5': 5.160783709475858}, 'constraint': array([ 0.72412598, -0.61396105,  1.00259381,  0.09472099])}

In [ ]:
sol = 'trust-constr'
list_realvalue = []
case_name = 'Case3'
runs_number = 100

def func(x):
    obj, c1, c2, c3, c4 = Case3(x)
    cons = [c1,c2,c3,c4]
    list_realvalue.extend([[obj]+cons])
    return obj,c1,c2,c3,c4

initial_x = []
fun_r = []
nfev_r = []
success_r = []
message_r = []
def recyc(x):
    nc_lb = np.array([-1,-1,-2,0,])
    nc_ub = np.array([2,  4, 3,1.5])
    cons = NonlinearConstraint(lambda x: func(x)[1:5], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x
    lb = [-5, -20,-10,-12,-15] 
    ub = [12,22,10,20,18] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0,  method='trust-constr', bounds=bd,constraints=cons)
    print(result.fun)
    initial_x.append(result.x)
    fun_r.append(result.fun)
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    f = xlwt.Workbook('encoding = utf-8') 
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True) 
    for i in range(len(initial_x)):
        x_values = initial_x[i].tolist()
        sheet1.write(i,0,x_values[0]) 
        sheet1.write(i,1,x_values[1])
        sheet1.write(i,2,fun_r[i])
        sheet1.write(i,3,nfev_r[i])
        sheet1.write(i,4,success_r[i])
        sheet1.write(i,5,message_r[i])
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')
    return result.x

def initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18] ,a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

x_random = initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18])
from itertools import product
for combination in product(x_random):
    recyc(combination)


C:\Users\fan_shuyuan\AppData\Local\Temp\ipykernel_45768\1804357572.py:26: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(objective, x0,  method='trust-constr', bounds=bd,constraints=cons)


16.19991213385418
9.669340646269298
9.6693406462693


D:\something\anaconda\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


1.0695605577606
11.17763579843486
9.669340646269298
11.177635798434855
1.0695605578279452
14.604818660820978
1.069560558175319
11.177635798434867
5.923409590029394
15.458867926153784
14.604818660820973
9.6693406462693
5.92340959002939
9.669340646269294
9.6693406462693
12.485759685246654
7.929909251990146
11.177635798434865
12.222316518418605
1.0695605578135603
13.620449145991804
5.923409590029394
16.19991213385418
7.929909251990148
5.9296545906183376
13.620449145991808
2.5264841137394893
9.669340646269298
9.669340646269298
16.239817307080372
16.842949221984355
9.669340646269296
3.6079991132978084
3.6079991132978084
11.177639923758917
13.620449145991802
5.923409590029394
5.923409590029394
9.669340646269303
16.065618875810724
17.400979446455267
13.620449145991808
3.6079991132978653
13.620449145991802
1.0695605580362764
12.485759685246647
17.579464941658234
7.92990925199015
15.458867926153793
9.669340646269298
3.6090631927145336
7.929909251990146
15.458867926153784
17.40097467732522
15.45

In [ ]:
sol = 'slsqp'
list_realvalue = []
case_name = 'Case3'
runs_number = 100

def func(x):
    obj, c1, c2, c3, c4 = Case3(x)
    cons = [c1,c2,c3,c4]
    list_realvalue.extend([[obj]+cons])
    return obj,c1,c2,c3,c4

initial_x = []
fun_r = []
nfev_r = []
success_r = []
message_r = []
def recyc(x):
    nc_lb = np.array([-1,-1,-2,0,])
    nc_ub = np.array([2,  4, 3,1.5])
    cons = NonlinearConstraint(lambda x: func(x)[1:5], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x
    lb = [-5, -20,-10,-12,-15] 
    ub = [12,22,10,20,18] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0,  method='slsqp', bounds=bd,constraints=cons)
    print(result.fun)
    initial_x.append(result.x)
    fun_r.append(result.fun)
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    f = xlwt.Workbook('encoding = utf-8') 
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True) 
    for i in range(len(initial_x)):
        x_values = initial_x[i].tolist()
        sheet1.write(i,0,x_values[0]) 
        sheet1.write(i,1,x_values[1])
        sheet1.write(i,2,fun_r[i])
        sheet1.write(i,3,nfev_r[i])
        sheet1.write(i,4,success_r[i])
        sheet1.write(i,5,message_r[i])
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')
    return result.x

def initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18] ,a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

x_random = initial_random(lb=[-5, -20,-10,-12,-15],ub=[12,22,10,20,18])
from itertools import product
for combination in product(x_random):
    recyc(combination)


C:\Users\fan_shuyuan\AppData\Local\Temp\ipykernel_45768\826562587.py:26: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(objective, x0,  method='slsqp', bounds=bd,constraints=cons)
D:\something\anaconda\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "


7.929909307435473
5.923409681485275
12.485759920814465
9.669340680149482
14.604818672032152
16.199912145962827
14.604818670790783
11.177635798434869
9.669340646269301
3.607999128180549
1.0695605650138558
12.485759908272493
5.92340959002944
3.607999113314879
11.177635798487445
16.199912133854095
12.48575968525544
14.60481866159429
11.177635813828852
7.929909260063351
7.929909330117933
1.069560578827304
3.6079991135267844
3.6079991224080428
1.0695605667161883
1.069560656527703
9.669340646286342
9.669340646324992
3.6079993317827967
5.923409640466765
3.6079991230476414
5.9234095900467985
16.19991213385409
17.400974687943666
1.069560561413049
1.0695605592849273
1.0695610706179477
1.0695606206292152
14.60481866093303
5.923409714505812
3.607999113423411
11.177635904450964
15.458867926383581
16.199912135652884
14.604818660820927
3.6079991180481525
17.40097468550895
3.6079991133602864
16.19991213589074
13.620449159427125
11.177635984193902
11.177635798476022
11.177635798661855
1.069560683074709

In [ ]:
sol = 'CBO'
case_name = 'Case3'
list_realvalue = []
runs_number = 100
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

def func_obj(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3(x)
    cons = [c1,c2,c3,c4]
    # print(('cons, obj value=', cons, obj))
    return -obj

def func_cons(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3(x)
    cons = [c1,c2,c3,c4]
    return cons

nc_lb = np.array([-1,-1,-2,0,])
nc_ub = np.array([2,  4, 3,1.5])
cons = NonlinearConstraint(func_cons, nc_lb, nc_ub)
pbounds = {'x1': (-5, 12),'x2': (-20, 22),'x3': (-10, 10),'x4': (-12, 20),'x5': (-15, 18),}
acquisition_function = UtilityFunction(kind="ucb", kappa=1)

for i in range(runs_number):
    optimizer = BayesianOptimization(f=func_obj, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
    optimizer.maximize(init_points=100, n_iter=200,acquisition_function=acquisition_function)
    print(optimizer.max)
    list_realvalue.extend([optimizer.max['target']])
    f = xlwt.Workbook('encoding = utf-8')
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
    for i in range(len(list_realvalue)):
        res = list_realvalue[i]
        sheet1.write(i,0,res)
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')


{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint': None}
{'target': None, 'params': None, 'constraint':

In [ ]:
sol = 'botorch'
case_name = 'Case3'
list_realvalue = []
iterations = []
max_values = []
runs_number = 100
itera = 0
import gpytorch
import torch
import tr_bo
from torch.quasirandom import SobolEngine
from botorch.utils.transforms import unnormalize
from botorch.models.model_list_gp_regression import ModelListGP

def func(x):
    global itera
    obj, c1, c2, c3, c4 = Case3(x)
    itera=itera+1

    return -obj,c1,c2, c3,c4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
dim = 1
batch_size = 4
n_init = 100 
max_cholesky_size = float("inf")  # Always use Cholesky

lb = torch.tensor([-1.0, -1, -2.0, 0,], dtype=torch.float64)
ub =  torch.tensor([2.0, 4, 3.0, 1.5, ], dtype=torch.float64)
bounds = torch.tensor([[-5, -20,  -10, -12, -15], [12, 22, 10, 20, 18]])

def objective(x):
    x_numpy = x.numpy()
    res = func(x_numpy)
    target = res[0]
    c1 = res[1]
    c2 = res[2]
    c3 = res[3]
    c4 = res[4]
    return target,c1,c2,c3,c4
def run_multiple_calculations(num_runs):
    for random_seed in range(num_runs):
        try:
            def eval_objective(x):
                return objective(unnormalize(x, bounds))
            # Initialize TuRBO state
            state = tr_bo.ScboState(dim, batch_size=batch_size)
            sobol = SobolEngine(dim, scramble=True, seed=1)

            # Get initial data
            # Must get initial values for both objective and constraints
            train_X = tr_bo.get_initial_points(dim, n_init,seed=random_seed)

            train_Y_tensor_list = []
            C1_tensor_list = []
            C2_tensor_list = []
            C3_tensor_list = []
            C4_tensor_list = []
            for x in train_X:
                result = eval_objective(x)
                combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                train_Y, C1, C2,C3,C4 = combined_tensor[0], combined_tensor[1], combined_tensor[2],combined_tensor[3], combined_tensor[4]
                train_Y_tensor_list.append(train_Y)
                C1_tensor_list.append(C1)
                C2_tensor_list.append(C2)
                C3_tensor_list.append(C3)
                C4_tensor_list.append(C4)

            train_Y = torch.stack(train_Y_tensor_list, dim=0)
            C1 = torch.stack(C1_tensor_list, dim=0)
            C2 = torch.stack(C2_tensor_list, dim=0)
            C3 = torch.stack(C3_tensor_list, dim=0)
            C4 = torch.stack(C4_tensor_list, dim=0)
            while not state.restart_triggered:  # Run until TuRBO converges
                # Fit GP models for objective and constraints
                model = tr_bo.get_fitted_model(train_X, train_Y,dim)
                c1_model = tr_bo.get_fitted_model(train_X, C1,dim)
                c2_model = tr_bo.get_fitted_model(train_X, C2,dim)
                c3_model = tr_bo.get_fitted_model(train_X, C3,dim)
                c4_model = tr_bo.get_fitted_model(train_X, C4,dim)
                # Generate a batch of candidates
                with gpytorch.settings.max_cholesky_size(max_cholesky_size):
                    X_next = tr_bo.generate_batch(
                        state=state,
                        model=model,
                        X=train_X,
                        Y=train_Y,
                        batch_size=batch_size,
                        n_candidates=2000,
                        constraint_model=ModelListGP(c1_model, c2_model,c3_model,c4_model),
                        sobol=sobol,
                    )

                Y_next_tensor_list = []
                C1_next_tensor_list = []
                C2_next_tensor_list = []
                C3_next_tensor_list = []
                C4_next_tensor_list = []
                for x in X_next:
                    result = eval_objective(x)
                    combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                    Y_next, C1_next, C2_next, C3_next, C4_next,= combined_tensor[0], combined_tensor[1], combined_tensor[2],combined_tensor[3], combined_tensor[4]
                    Y_next_tensor_list.append(Y_next)
                    C1_next_tensor_list.append(C1_next)
                    C2_next_tensor_list.append(C2_next)
                    C3_next_tensor_list.append(C3_next)
                    C4_next_tensor_list.append(C4_next)
                Y_next = torch.stack(Y_next_tensor_list, dim=0)
                C1_next = torch.stack(C1_next_tensor_list, dim=0)
                C2_next = torch.stack(C2_next_tensor_list, dim=0)
                C3_next = torch.stack(C3_next_tensor_list, dim=0)
                C4_next = torch.stack(C4_next_tensor_list, dim=0)

                C_next = torch.cat([C1_next, C2_next,C3_next, C4_next,], dim=-1)
                state = tr_bo.update_state(state, Y_next, C_next,ub,lb)
                train_X = torch.cat((train_X, X_next), dim=0)
                train_Y = torch.cat((train_Y, Y_next), dim=0)
                C1 = torch.cat((C1, C1_next), dim=0)
                C2 = torch.cat((C2, C2_next), dim=0)
                C3 = torch.cat((C3, C3_next), dim=0)
                C4 = torch.cat((C4, C4_next), dim=0)
                C = torch.cat([C1, C2, C3, C4], dim=-1)
                bool_tensor = (C >= lb.repeat(len(C), 1)) & (C <= ub.repeat(len(C), 1))
                bool_tensor = torch.all(bool_tensor, dim=-1)
                Valid_Y = train_Y[bool_tensor]
                if len(Valid_Y)==0:
                    break
                else:
                    list_realvalue.append(max(Valid_Y[0]))
                if len(list_realvalue) == 50:
                    max_value = max(list_realvalue).item()
                    print("Valid_Y",max_value)
                    max_values.append(max_value)
                    f = xlwt.Workbook('encoding = utf-8')
                    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
                    if len(iterations)==0:
                        iterations.append(itera)
                    else:
                        iterations.append(itera-sum(iterations))
                    for i in range(len(iterations)):
                        sheet1.write(i,0,max_values[i])
                        sheet1.write(i,1,iterations[i])
                    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')
                    list_realvalue.clear()
                    break
        except Exception as e:
            if len(list_realvalue)==0:
                max_value=None
            else:
                max_value = max(list_realvalue).item()
            print("Attempting another calculation...",max_value)
            max_values.append(max_value)
            f = xlwt.Workbook('encoding = utf-8')
            sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
            if len(iterations)==0:
                iterations.append(itera)
            else:
                iterations.append(itera-sum(iterations))
            for i in range(len(iterations)):
                sheet1.write(i,0,max_values[i])
                sheet1.write(i,1,iterations[i])
            f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')
            list_realvalue.clear()        
run_multiple_calculations(num_runs=runs_number)


Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calculation... None
Attempting another calcul

In [6]:
sol = 'PSO'
case_name = 'Case3'
list_realvalue = []
iterations=[]
runs_number = 100
itera = 0
from pyswarm import pso

def func(x):
    global itera
    obj, c1, c2, c3, c4 = Case3(x)
    cons = [c1+1,2-c1,c2+1,4-c2,c3+2,3-c3,c4,1.5-c4]
    global x_p
    try:
        equal = np.array_equal(x,x_p)
        if not equal:
            itera=itera+1
    except:
        itera=itera+1
    x_p =x
    return obj,cons

for num in range(runs_number):
    xopt, fopt = pso(lambda x:func(x)[0], lb =[-5, -20,-10,-2,-15] ,ub=[12,22,10,20,18],f_ieqcons=lambda x:func(x)[1:2], swarmsize=100, omega=0.5, phip=0.5, phig=0.5, maxiter=200, minstep=1e-8,minfunc=1e-8, debug=False)
    print('xopt, fopt',xopt, fopt)
    xopt = np.append(xopt,fopt)
    list_realvalue.extend([xopt])
    f = xlwt.Workbook('encoding = utf-8')
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
    if len(iterations)==0:
        iterations.append(itera)
    else:
        iterations.append(itera-sum(iterations))
    for i in range(len(list_realvalue)):
        x_values = list_realvalue[i]
        sheet1.write(i,0,x_values[5])
        sheet1.write(i,3,iterations[i])
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')


Stopping search: Swarm best position change less than 1e-08
xopt, fopt [ 4.17721194 10.95548088 -3.22903288  8.16847087  2.46903656] 10.392962990655128
Stopping search: Swarm best objective change less than 1e-08
xopt, fopt [ 3.40557056  7.98887163  2.15812653 12.28950728 -0.86520112] 9.702010660783179
Stopping search: Swarm best position change less than 1e-08
xopt, fopt [-0.14311491 15.09995883 -6.59475155  3.94909053  1.15467149] 1.7540618240741144
Stopping search: Swarm best position change less than 1e-08
xopt, fopt [ 3.30090456 -1.90355462 -0.06257951  8.60020165 -7.11610029] 9.32387725802963
Stopping search: maximum iterations reached --> 200
However, the optimization couldn't find a feasible design. Sorry
xopt, fopt [  8.83969178 -19.26459628   7.87231694  15.85372815  -0.34746016] 1e+100
Stopping search: Swarm best position change less than 1e-08
xopt, fopt [-0.35825807  8.70886466  3.85544773 15.37918022  3.3589048 ] 2.9761759029569217
Stopping search: maximum iterations reac

In [ ]:
sol = 'GA'
case_name = 'Case3'
list_realvalue = []
iterations=[]
runs_number = 100
itera = 0
from sko.GA import GA
from sko.operators import crossover

def func(x):
    global itera
    obj, c1, c2, c3, c4 = Case3(x)
    cx1 = -c1-1
    cx11=c1-2
    cx2=-1-c2
    cx22=c2-4
    cx3=-2-c3
    cx33=c3-3
    cx4=-c4
    cx44=c4-1.5
    itera=itera+1
    return obj,cx1,cx2,cx3,cx4,cx11,cx22,cx33,cx44


for num in range(runs_number):
    ga = GA(func=lambda x:func(x)[0], n_dim=5, size_pop=100, max_iter=200,lb =[-5, -20,-10,-2,-15] ,ub=[12,22,10,20,18], precision=1e-7,prob_mut = 0.001, constraint_ueq=[lambda x:func(x)[i] for i in range(1,9)])
    ga.register(operator_name='crossover', operator=crossover.crossover_2point_prob, crossover_prob=0.5)
    best_x, best_y = ga.run()
    best_res = np.concatenate((best_y, best_x))
    print('best_res',best_res)
    list_realvalue.extend([best_res])
    f = xlwt.Workbook('encoding = utf-8')
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
    if len(iterations)==0:
        iterations.append(itera)
    else:
        iterations.append(itera-sum(iterations))
    for i in range(len(list_realvalue)):
        x_values = list_realvalue[i]
        sheet1.write(i,0,x_values[0])
        sheet1.write(i,3,iterations[i])
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')


best_res [ 1.06956084e+00  1.00582839e-07 -1.97354381e+01 -8.32676090e+00
  1.70724147e+01  3.49799263e+00]
best_res [ 1.06956063e+00 -2.60770321e-08 -1.65809978e+01 -1.50525589e+00
  2.82342874e+00  4.63111225e+00]
best_res [ 1.06956063e+00 -2.60770321e-08 -1.49392786e+01  9.37485896e+00
  1.20619852e+01  4.32972436e+00]
best_res [ 1.06956063e+00 -2.60770321e-08 -9.05465218e+00 -7.79348507e+00
  1.32707492e+01  5.14783783e+00]
best_res [ 1.06956063e+00 -2.60770321e-08 -1.15265195e+01  1.45933666e-01
  8.34397482e+00 -2.98715548e+00]
best_res [ 1.06956063e+00 -2.60770321e-08  7.96295878e+00  1.74269204e+00
  1.89936601e+01 -1.88014151e+00]
best_res [ 1.06956228e+00  6.07222321e-07 -1.69881206e+01 -9.68558892e+00
  1.57257654e-01  7.78103506e+00]
best_res [ 1.06956063e+00 -2.60770321e-08  5.12046412e+00  3.54032365e+00
  1.60888838e+01  2.70929649e+00]
best_res [ 1.06956063e+00 -2.60770321e-08 -1.36827032e+01 -7.38298631e+00
  1.01431131e+01 -1.13036355e+01]
best_res [ 1.06956063e+00 -2